In [64]:
import pandas as pd
import numpy as np
import re
import networkx as nx
import matplotlib.pyplot as plt

In [65]:
from collections import Counter

In [71]:
magazine_d = pd.read_csv('./magazine/final/magazine_Di.csv', sep='|', encoding = 'utf-8')

In [72]:
magazine_d.head()

,Unnamed: 0,magazine,brand,category,title,link,date,contents,title_brand,check,word_dior,sentences_tag,ADFN,item
0,0,A,D,FASHION,빈티지 쇼핑의 시대,http://www.allurekorea.com/2020/10/27/%eb%b9%8...,2020-10-27,"['20년이 지난 패션템에 새로운 활기가 돈다. 바야흐로 빈티지 쇼핑의 시대다.',...",0,0,"['그러니 빈티지 패션은, 어쩌면 나도 좋고 너도 좋은 일이다. 당시 사랑했던, 지...","[('[', 'SS'), (""'"", 'SS'), ('그러', 'VV'), ('니',...","['빈티지', '패션', '좋', '느', '좋', '당시', '사랑', '사랑',...","['옷', '데님', '재킷', '재킷', '재킷', '팬츠', '가방', 'DIOR']"
1,1,A,D,FASHION,"주식을 시작한 패션 에디터의 이야기, 나는 주주다",http://www.allurekorea.com/2020/10/16/%ec%a3%b...,2020-10-16,['주식의 ‘주’자도 모르는 패션 에디터가 주식에 눈뜨려 하는 순간. 이번 시즌 가...,0,0,[' 요즘 들어 주식은 친구들과의 대화에서 빼놓을 수 없는 주제가 되었다. 코로나...,"[('[', 'SS'), (""'"", 'SS'), ('요즘', 'NNG'), ('듣'...","['요즘', '주식', '친구', '과의', '대화', '빼놓', '주제', '코로...","['DIOR', '가방', '가방', '가방', 'DIOR']"
2,2,A,D,FASHION,거리로 나온 하이패션 슈즈들 4,http://www.allurekorea.com/2020/09/16/%eb%a6%a...,2020-09-16,['나이키의 조던 시리즈들 중에서도 역대급으로 평가받는 ‘에어 디올’은 부르는 게 ...,0,0,"[""['나이키의 조던 시리즈들 중에서도 역대급으로 평가받는 ‘에어 디올’은 부르는 ...","[('[', 'SS'), ('""', 'SS'), ('[', 'SS'), (""'"", ...","['나이', '조', '덜', '시리즈', '역대', '급', '평가', '에어',...","['DIOR', '슈즈', 'DIOR', 'DIOR', 'DIOR']"
3,3,A,D,FASHION,차정원부터 아이린까지. 옷 잘입는 셀럽들의 사복 패션에 등장한 뉴 백은 무엇?,http://www.allurekorea.com/2020/08/13/%ec%b0%a...,2020-08-13,"['셀럽들의 데일리룩에서 캐치한 뉴 백!', '@ch_amii', '지갑, 핸드폰,...",0,0,['디올 레이디 디올 디올아무르 미니백 5백만원대'],"[('[', 'SS'), (""'"", 'SS'), ('DIOR', 'OL'), ('L...","['DIOR', 'LADY', 'DIOR', 'DIOR', '아무르', '미니']","['DIOR', 'DIOR', 'DIOR']"
4,4,A,D,CELEBRITY,DON’T CATEGORIZE ME / 유태오,http://www.allurekorea.com/2020/05/28/dont-cat...,2020-05-28,['“Don’t Categorize Me.” 그가 말했다. 배우 유태오는 더없이 자...,0,0,['실크 슬리브리스와 블랙 와이드 팬츠는 김서룡 옴므. 블랙 부츠는 보테가 베네타....,"[('[', 'SS'), (""'"", 'SS'), ('실크', 'NNG'), ('슬리...","['실크', '슬리브리스', '블랙', '와이드', '팬츠', '김', '룡', '...","['팬츠', '부츠', 'DIOR', 'DIOR']"


In [73]:
len(magazine_d)

1759

In [74]:
mag_d = magazine_d[['magazine', 'brand','title', 'link', 'date', 'word_dior','ADFN']]

In [75]:
mag_d.head()

,magazine,brand,title,link,date,word_dior,ADFN
0,A,D,빈티지 쇼핑의 시대,http://www.allurekorea.com/2020/10/27/%eb%b9%8...,2020-10-27,"['그러니 빈티지 패션은, 어쩌면 나도 좋고 너도 좋은 일이다. 당시 사랑했던, 지...","['빈티지', '패션', '좋', '느', '좋', '당시', '사랑', '사랑',..."
1,A,D,"주식을 시작한 패션 에디터의 이야기, 나는 주주다",http://www.allurekorea.com/2020/10/16/%ec%a3%b...,2020-10-16,[' 요즘 들어 주식은 친구들과의 대화에서 빼놓을 수 없는 주제가 되었다. 코로나...,"['요즘', '주식', '친구', '과의', '대화', '빼놓', '주제', '코로..."
2,A,D,거리로 나온 하이패션 슈즈들 4,http://www.allurekorea.com/2020/09/16/%eb%a6%a...,2020-09-16,"[""['나이키의 조던 시리즈들 중에서도 역대급으로 평가받는 ‘에어 디올’은 부르는 ...","['나이', '조', '덜', '시리즈', '역대', '급', '평가', '에어',..."
3,A,D,차정원부터 아이린까지. 옷 잘입는 셀럽들의 사복 패션에 등장한 뉴 백은 무엇?,http://www.allurekorea.com/2020/08/13/%ec%b0%a...,2020-08-13,['디올 레이디 디올 디올아무르 미니백 5백만원대'],"['DIOR', 'LADY', 'DIOR', 'DIOR', '아무르', '미니']"
4,A,D,DON’T CATEGORIZE ME / 유태오,http://www.allurekorea.com/2020/05/28/dont-cat...,2020-05-28,['실크 슬리브리스와 블랙 와이드 팬츠는 김서룡 옴므. 블랙 부츠는 보테가 베네타....,"['실크', '슬리브리스', '블랙', '와이드', '팬츠', '김', '룡', '..."


In [76]:
mag_d.loc[0, 'ADFN'][2]

'빈'

In [77]:
mag_d['ADFN_2'] = mag_d['ADFN']

C:\Users\wnsgo\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [81]:
len(mag_d)

1759

In [80]:
mag_d.to_csv('./magazine/final/magazine_Dior.csv', sep = '|', encoding = 'utf-8')

### 쪼개진 단어 붙이기

#### 사전에 등록했으면 하는 단어
['레디 투 웨어', '공방', '트위드', 'FW', 'F/W', 'SS', 'S/S', '오간자', '아름다움', '그랑 팔레', '캐주얼', '런웨이', 
 '오프숄더', '레이어드', '인스타그램', '스타일링']

In [101]:
for word in mag_d['ADFN_2']:
    if "비주" in word:
        print(word)
        print('-'*50)

['패션', '전시', '특정', '공간', 'SPACE', '패션', '오브', 'FASHION', 'OBJET', '메시지', 'MESSAGE', '조화', '어우러지', '스토리', 'STORY', '작업', '건축', '요소', '작품', '옷', '관람자', '동선', '따르', '감정', '흐름', '변화', '빛', '그림자', '음향', '효과', '정확', '계산', '점', '홍', '콩', '세계', '순회', '샤넬', '모바일', '아트', '파', '빌', 'MOBILE', 'ART', 'PAVILION', '서울', '경희궁', '열리', '프라다', '웨이스트', '다운', 'WAIST', 'DOWN', '뉴욕', '메트로폴리탄', '뮤지엄', 'ALEXANDER', '맥', '퀸', '새', '비지', '뷰티', 'ALEXANDER', 'MCQUEEN', 'SAVAGE', 'BEAUTY', '모스크', '푸슈킨', '미술관', '인스', '레이션', 'DIOR', 'INSPIRATION', 'DIOR', '패션', '전시', '세기', '문화', '예술', '분야', '이슈', '패션', '전시', '동시대', '화두', '담', '움직이', '변형', '건축', '실험', '정', '신', '천재', '디자이너', '그리움', '존경', '감정', '표현', '옷', '역사', '감동', '미래', '문화', '작업', '메트로폴리탄', '뮤', '지', 'ALEXANDER', '맥', '퀸', '전시', '기획', '총괄', '큐레이터', '헤롤드', '코다', '패션', '전시', '가지', '의미', '이렇', '이야기', '생', '존', '필수', '조건', '시작하', '의복', '예술', '어깨', '경지', '렀', '패션', '디자이너', '개인적', '취향', '넘', '문화', '경제', '정치적', '의미', '지니', '사회', '영향', '미치', '의미', '개념적', '전시', '관람객', '자유',

In [97]:
# 테일러 / 레이어
mag_d['ADFN_2'] = mag_d['ADFN_2'].str.replace("'테일러', '드'", "테일러드")
mag_d['ADFN_2'] = mag_d['ADFN_2'].str.replace("'레이어', '드'", "레이어드")

mag_d['ADFN_2'] = mag_d['ADFN_2'].str.replace("'테일러', '링'", "테일러링")
mag_d['ADFN_2'] = mag_d['ADFN_2'].str.replace("'레이어', '링'", "레이어링")

mag_d['ADFN_2'] = mag_d['ADFN_2'].str.replace("'에스', '닉'", "에스닉")

C:\Users\wnsgo\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\wnsgo\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\wnsgo\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

In [102]:
mag_d.to_csv('./magazine/final/magazine_d_stickword.csv', sep='|', encoding ='utf-8', index=False)